# Neo4j와 LangChain을 활용한 영화 추천 시스템

---

## 1. Neo4J AuraDB 환경 설정

In [1]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

True

In [2]:
from langchain_neo4j import Neo4jGraph

# LangChain 도구 활용 - DB 연결 객체 초기화 
graph = Neo4jGraph( 
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
    enhanced_schema=True, # LLM이 많은 정보를 확인하기 위한 확장 스키마 출력 설정
)

In [3]:
# 테스트 쿼리 실행 
cypher_query = """
MATCH (n:Movie)
RETURN COUNT(n) AS Movie_Count
"""

graph.query(cypher_query)

[{'Movie_Count': 4803}]

---

## 2. Text-to-Cypher 활용

Text-to-Cypher는 자연어 질의를 Neo4j 데이터베이스 쿼리 언어인 Cypher로 변환하여 지식 그래프를 효과적으로 검색할 수 있는 기술입니다. 영화 데이터베이스에서 사용자 질문을 Neo4j Cypher 쿼리로 변환하는 방법을 배우게 됩니다. 

- **학습 목표**:

   - LLM을 활용하여 자연어 질의를 Cypher 쿼리로 변환하는 방법 이해
   - `GraphCypherQAChain`의 사용법 익히기
   - 다양한 영화 데이터 쿼리 패턴 학습
   - 커스텀 프롬프트를 활용한 정교한 쿼리 생성 방법 습득

- **필요 사항**:
   - Neo4j 데이터베이스 (영화 데이터 포함)
   - 필요 패키지: `langchain-neo4j`, `langchain-openai`

### 2.1 영화 데이터베이스 스키마 확인

영화 데이터베이스는 다음과 같은 노드와 관계로 구성되어 있습니다:

- **노드 속성:**
   - **Person {id: STRING, name: STRING}**: 영화 관련 인물 정보
   - **Movie {id: STRING, title: STRING, released: STRING, rating: FLOAT, overview: STRING, runtime: INTEGER, tagline: STRING, content_embedding: LIST}**: 영화 정보
   - **Genre {id: STRING, name: STRING}**: 영화 장르 정보

- **관계:**
   - **(:Person)-[:ACTED_IN]->(:Movie)**: 배우와 영화 간의 출연 관계
   - **(:Person)-[:DIRECTED]->(:Movie)**: 감독과 영화 간의 감독 관계
   - **(:Movie)-[:IN_GENRE]->(:Genre)**: 영화와 장르 간의 분류 관계

In [4]:
graph.refresh_schema() # 스키마 새로고침
print(graph.schema) # 스키마 출력

Node properties:
- **Person**
  - `id`: STRING Example: "person-D.W. Griffith"
  - `name`: STRING Example: "D.W. Griffith"
- **Movie**
  - `id`: STRING Example: "movie-4592"
  - `rating`: FLOAT Min: 0.0, Max: 10.0
  - `title`: STRING Example: "Intolerance"
  - `released`: STRING Example: "1916-09-04"
  - `overview`: STRING Example: "The story of a poor young woman, separated by prej"
  - `runtime`: INTEGER Min: 0, Max: 338
  - `tagline`: STRING Example: "The Cruel Hand of Intolerance"
- **Genre**
  - `id`: STRING Example: "genre-Drama"
  - `name`: STRING Example: "Drama"
Relationship properties:

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


### 2.2 LangChain을 활용한 Text-to-Cypher 구현

- LangChain으로 Neo4J 지식 그래프 조회
- LLM을 사용하여 사용자 질의를 Cypher 쿼리로 변환

#### 1) **GraphCypherQAChain** 설정

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_neo4j import GraphCypherQAChain

# LLM 및 그래프 객체 초기화
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.0)

# GraphCypherQAChain 객체 초기화 (자연어 질의를 Cypher 쿼리로 변환하고 실행하는 체인)
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,                            # llm: 질의를 Cypher로 변환할 LLM 모델
    graph=graph,                        # graph: 쿼리를 실행할 Neo4j 그래프 객체
    allow_dangerous_requests=True,      # allow_dangerous_requests=True: 위험한 요청 허용 설정
    verbose=True,                       # verbose=True: 디버깅을 위한 상세 로그 출력    

    return_direct=False                 # ★★★return_direct=True: Cypher 쿼리 결과를 직접 반환, False: LLM이 Cypher 쿼리 결과를 바탕으로 자연어 답변 생성
)

#### 2) **기본 영화 쿼리** 

- Text to Cypher (DB 조회)

In [13]:
# 영화 제목으로 정보 검색
answer = cypher_chain.invoke({"query": "영화 'Apollo 13'에 대한 정보를 알려주세요."})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Apollo 13"})
OPTIONAL MATCH (m)<-[:ACTED_IN]-(p:Person)
OPTIONAL MATCH (m)<-[:DIRECTED]-(d:Person)
OPTIONAL MATCH (m)-[:IN_GENRE]->(g:Genre)
RETURN m, collect(DISTINCT p) AS actors, collect(DISTINCT d) AS directors, collect(DISTINCT g) AS genres
Full Context:
[{'m': {'overview': 'The true story of technical troubles that scuttle the Apollo 13 lunar mission in 1971, risking the lives of astronaut Jim Lovell and his crew, with the failed journey turning into a thrilling saga of heroism. Drifting more than 200,000 miles from Earth, the astronauts work furiously with the ground crew to avert tragedy.', 'rating': 7.3, 'runtime': 140, 'tagline': 'Houston, we have a problem.', 'content_embedding': [-0.04420690983533859, 0.01735256053507328, 0.040657781064510345, 0.02014918625354767, 0.008586429059505463, -0.03241390734910965, 0.03409862518310547, 0.004922174382954836, 0.015128737315535545, 0.017318865284323

In [14]:
# 답변 출력
print(answer)

{'query': "영화 'Apollo 13'에 대한 정보를 알려주세요.", 'result': '영화 \'Apollo 13\'은 1995년 6월 30일에 개봉한 드라마 장르의 작품입니다. 이 영화는 1971년 아폴로 13호 달 탐사 임무 중 발생한 기술적 문제로 인해 우주비행사 짐 러벨과 그의 승무원들의 생명이 위태로워진 실화를 바탕으로 하고 있습니다. 지구에서 20만 마일 이상 떨어진 우주에서 우주비행사들과 지상 관제팀이 협력하여 비극을 막기 위해 필사적으로 노력하는 긴장감 넘치는 영웅담을 그립니다. 영화의 러닝타임은 140분이며, 평점은 7.3점입니다. 유명 배우로는 톰 행크스, 케빈 베이컨, 에드 해리스, 빌 팩스턴, 게리 시니스가 출연했으며, 감독은 론 하워드입니다. 영화의 대표적인 태그라인은 "Houston, we have a problem."입니다.'}


In [11]:
# LLM 답변 출력
from pprint import pprint
pprint(answer['result'])

# 전체 출력 (디버깅 정보 포함)

[{'m.id': 'movie-635',
  'm.overview': 'The true story of technical troubles that scuttle the Apollo '
                '13 lunar mission in 1971, risking the lives of astronaut Jim '
                'Lovell and his crew, with the failed journey turning into a '
                'thrilling saga of heroism. Drifting more than 200,000 miles '
                'from Earth, the astronauts work furiously with the ground '
                'crew to avert tragedy.',
  'm.rating': 7.3,
  'm.released': '1995-06-30',
  'm.runtime': 140,
  'm.tagline': 'Houston, we have a problem.',
  'm.title': 'Apollo 13'}]


In [30]:
# 특정 감독의 영화 찾기
answer = cypher_chain.invoke({"query": "'Christopher Nolan' 감독의 영화를 모두 찾아주세요."})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person {name: "Christopher Nolan"})-[:DIRECTED]->(m:Movie)
RETURN m

Full Context:
[{'m': {'overview': "Suffering short-term memory loss after a head injury, Leonard Shelby embarks on a grim quest to find the lowlife who murdered his wife in this gritty, complex thriller that packs more knots than a hangman's noose. To carry out his plan, Shelby snaps Polaroids of people and places, jotting down contextual notes on the backs of photos to aid in his search and jog his memory. He even tattoos his own body in a desperate bid to remember.", 'rating': 8.1, 'runtime': 113, 'tagline': 'Some memories are best forgotten.', 'content_embedding': [0.0044687893241643906, 0.05293796584010124, -0.07254564017057419, 0.012423234060406685, -0.018700163811445236, -0.0042281621135771275, 0.0440828874707222, 0.012925113551318645, -0.016513895243406296, -0.02718398906290531, 0.003949722275137901, -0.03800533339381218, -0.04958293

In [31]:
# 답변 출력
print(answer)

{'query': "'Christopher Nolan' 감독의 영화를 모두 찾아주세요.", 'result': 'Christopher Nolan 감독의 영화는 다음과 같습니다: Memento, Insomnia, Batman Begins, The Prestige, The Dark Knight, Inception, The Dark Knight Rises, Interstellar.'}


In [32]:
# LLM 답변 출력
pprint(answer['result'])

('Christopher Nolan 감독의 영화는 다음과 같습니다: Memento, Insomnia, Batman Begins, The '
 'Prestige, The Dark Knight, Inception, The Dark Knight Rises, Interstellar.')


In [33]:
# 특정 배우가 출연한 영화 찾기
answer = cypher_chain.invoke({"query": "'Tom Hanks'가 출연한, 평점이 가장 높은 영화 3개는 무엇인가요?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie)
RETURN m.title, m.rating
ORDER BY m.rating DESC
LIMIT 3

Full Context:
[{'m.title': 'Forrest Gump', 'm.rating': 8.2}, {'m.title': 'The Green Mile', 'm.rating': 8.2}, {'m.title': 'Saving Private Ryan', 'm.rating': 7.9}]

> Finished chain.


In [34]:
# 답변 출력
print(answer)

{'query': "'Tom Hanks'가 출연한, 평점이 가장 높은 영화 3개는 무엇인가요?", 'result': 'Tom Hanks가 출연한 평점이 가장 높은 영화 3개는 Forrest Gump, The Green Mile, 그리고 Saving Private Ryan입니다. 각각의 평점은 8.2, 8.2, 7.9입니다.'}


In [35]:
# LLM 답변 출력
pprint(answer['result'])

('Tom Hanks가 출연한 평점이 가장 높은 영화 3개는 Forrest Gump, The Green Mile, 그리고 Saving '
 'Private Ryan입니다. 각각의 평점은 8.2, 8.2, 7.9입니다.')


In [36]:
# 장르별 영화 검색
answer = cypher_chain.invoke({"query": "2010년 이후 개봉한 'Drama' 장르 영화 중 평점이 높은 순서로 5개를 보여주세요."})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre {name: "Drama"})
WHERE toInteger(substring(m.released, 0, 4)) > 2010
RETURN m
ORDER BY m.rating DESC
LIMIT 5

Full Context:
[{'m': {'overview': 'A womanizing yet lovable loser, Charlie, a waiter in his early 30\'s who dreams of selling his book entitled "7 STEPS OF HEALING THE MALE BROKEN HEART" finds himself still working in restaurants to survive in the Big Apple. Low on cash, he\'s left with no other choice but to look for a roommate to share his tiny studio. Surprisingly, the first person to answer the ad is his ex and only love of his life Pam, who broke his heart and disappeared without reason and the inspiration behind his book. The Pam he remembered was a youthful spirit with lots of money who is now broke and disheveled. A new story begins and it is up to Charlie to find out why she ran out on him and what\'s happened to her over the past three years. With a potential ne

In [37]:
# 답변 출력
print(answer)

{'query': "2010년 이후 개봉한 'Drama' 장르 영화 중 평점이 높은 순서로 5개를 보여주세요.", 'result': '2010년 이후 개봉한 \'Drama\' 장르 영화 중 평점이 높은 순서로 5개는 다음과 같습니다:\n\n1. "Me You and Five Bucks" (2015-07-07) - 평점 10.0  \n2. "Whiplash" (2014-10-10) - 평점 8.3  \n3. "Guten Tag, Ramón" (2013-10-18) - 평점 8.1  \n4. "Interstellar" (2014-11-05) - 평점 8.1  \n5. "Room" (2015-10-16) - 평점 8.1  \n\n이 영화들은 모두 2010년 이후에 개봉되었으며, 평점이 높은 드라마 장르 영화입니다.'}


In [38]:
# LLM 답변 출력
pprint(answer['result'])

("2010년 이후 개봉한 'Drama' 장르 영화 중 평점이 높은 순서로 5개는 다음과 같습니다:\n"
 '\n'
 '1. "Me You and Five Bucks" (2015-07-07) - 평점 10.0  \n'
 '2. "Whiplash" (2014-10-10) - 평점 8.3  \n'
 '3. "Guten Tag, Ramón" (2013-10-18) - 평점 8.1  \n'
 '4. "Interstellar" (2014-11-05) - 평점 8.1  \n'
 '5. "Room" (2015-10-16) - 평점 8.1  \n'
 '\n'
 '이 영화들은 모두 2010년 이후에 개봉되었으며, 평점이 높은 드라마 장르 영화입니다.')


#### 3) **출력 갯수를 지정 (top k)** 

In [54]:
# top_k 파라미터로 결과 수 제한
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph, 
    allow_dangerous_requests=True,
    verbose=True,
    top_k=5,  # 최대 5개의 결과만 반환
)

answer = cypher_chain.invoke({"query": "'Tom Hanks' 배우가 출연한 영화를 모두 알려주세요."})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie) RETURN m
Full Context:
[{'m': {'overview': "A successful businessman falls in love with the girl of his dreams. There's one big complication though; he's fallen hook, line and sinker for a mermaid.", 'rating': 6.1, 'runtime': 111, 'tagline': "Allen Bauer Thought He'd Never Find The Right Woman... He Was Only Half Wrong!", 'content_embedding': [-0.013964681886136532, 0.021746326237916946, 0.015536865219473839, 0.03408597782254219, 0.009175470098853111, -0.02489069104194641, -0.0024094691034406424, 0.010179553180932999, -0.024150840938091278, -0.029726143926382065, -0.017809264361858368, -0.06880611926317215, -0.01566898077726364, -0.021244285628199577, -0.006206158548593521, 0.0667451024055481, 0.009935137815773487, -0.06056206673383713, -0.02129713073372841, 0.029620451852679253, 0.014849861152470112, -0.007768432609736919, -0.009518972598016262, -0.014110010117292

In [44]:
# LLM 답변 출력
pprint(answer['result'])

("Tom Hanks 배우가 출연한 영화는 'Bridge of Spies', 'Saving Mr. Banks', 그리고 'Captain "
 "Phillips'입니다.")


#### 4) **중간 과정 확인** 

- `return_intermediate_steps`=True

In [55]:
# 중간 결과를 포함하여 출력
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph, 
    allow_dangerous_requests=True,
    verbose=True,
    return_intermediate_steps=True  # 생성된 Cypher 쿼리와 중간 결과 확인
)

# 평점 8점 이상인 액션 영화 찾기
answer = cypher_chain.invoke({"query": "평점이 8점 이상인 'Action' 장르 영화는 무엇이 있나요?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre {name: "Action"})
WHERE m.rating >= 8.0
RETURN m.title, m.rating
Full Context:
[{'m.title': 'Seven Samurai', 'm.rating': 8.2}, {'m.title': 'Star Wars', 'm.rating': 8.1}, {'m.title': 'The Empire Strikes Back', 'm.rating': 8.2}, {'m.title': 'Scarface', 'm.rating': 8.0}, {'m.title': "One Man's Hero", 'm.rating': 9.3}, {'m.title': 'The Lord of the Rings: The Fellowship of the Ring', 'm.rating': 8.0}, {'m.title': 'The Lord of the Rings: The Two Towers', 'm.rating': 8.0}, {'m.title': 'Oldboy', 'm.rating': 8.0}, {'m.title': 'The Lord of the Rings: The Return of the King', 'm.rating': 8.1}, {'m.title': 'Star Wars: Clone Wars: Volume 1', 'm.rating': 8.0}]

> Finished chain.


In [46]:
# 중간 결과 확인
for k, v in answer.items():
    print(f"{k}: {v}")

query: 평점이 8점 이상인 'Action' 장르 영화는 무엇이 있나요?
result: 평점이 8점 이상인 'Action' 장르 영화로는 'The Dark Knight' (평점 8.2), 'Inception' (평점 8.1), 그리고 'The Empire Strikes Back' (평점 8.2) 등이 있습니다.
intermediate_steps: [{'query': 'cypher\nMATCH (m:Movie)-[:IN_GENRE]->(g:Genre {name: "Action"})\nWHERE m.rating >= 8.0\nRETURN m.title AS title, m.rating AS rating\nORDER BY m.rating DESC\n'}, {'context': [{'title': "One Man's Hero", 'rating': 9.3}, {'title': 'Seven Samurai', 'rating': 8.2}, {'title': 'The Empire Strikes Back', 'rating': 8.2}, {'title': 'The Dark Knight', 'rating': 8.2}, {'title': 'Star Wars', 'rating': 8.1}, {'title': 'The Lord of the Rings: The Return of the King', 'rating': 8.1}, {'title': 'Inception', 'rating': 8.1}, {'title': 'Scarface', 'rating': 8.0}, {'title': 'The Lord of the Rings: The Fellowship of the Ring', 'rating': 8.0}, {'title': 'The Lord of the Rings: The Two Towers', 'rating': 8.0}]}]


#### 5) **직접 Cypher 결과 얻기 (LLM 답변 없이)** 

- `return_direct`=True

In [ ]:
# 직접 Cypher 결과 얻기 (LLM 답변 없이)
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph, 
    allow_dangerous_requests=True,
    verbose=True,
    return_direct=True  # LLM 답변 생성 단계 건너뛰기
)

# 2000년대 개봉한 영화 중 평점 순 정렬
answer = cypher_chain.invoke({"query": "2000년대(2000-01-01 ~ 2009-12-31) 개봉한 영화를 평점 순으로 정렬해주세요."})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)
WHERE m.released >= "2000-01-01" AND m.released <= "2009-12-31"
RETURN m
ORDER BY m.rating DESC


> Finished chain.


In [48]:
for k, v in answer.items():
    print(f"{k}: {v}")

query: 2000년대(2000-01-01 ~ 2009-12-31) 개봉한 영화를 평점 순으로 정렬해주세요.
result: [{'m': {'overview': 'An aging out of work clown returns to his small hometown, resigned to spend the rest of his days in a drunken stupor. But when his passion for clowning is reawakened by the local amateur circus he finds his smile.', 'rating': 10.0, 'runtime': 0, 'content_embedding': [-0.024030201137065887, 0.033846501260995865, -0.007876187562942505, 0.028618402779102325, -0.04582756757736206, -0.011456891894340515, 0.04201541095972061, 0.0075017791241407394, -0.024275267496705055, 0.0046222396194934845, 0.01302940584719181, -0.05723680555820465, -0.048686683177948, 0.049040667712688446, 0.010272400453686714, 0.04098068177700043, 0.0017044083215296268, 0.027188844978809357, -0.016787102445960045, 0.06600476801395416, 0.03041556291282177, -0.014309201389551163, 0.013294895179569721, -0.04375810921192169, 0.03803987428545952, -0.005878209136426449, -0.04748857766389847, 0.006923148408532143, 0.07439150661230087, 0.

In [49]:
import pandas as pd
pd.DataFrame([item['m'] for item in answer['result']])

,overview,rating,runtime,content_embedding,id,title,released,tagline
0,An aging out of work clown returns to his smal...,10.0,0,"[-0.024030201137065887, 0.033846501260995865, ...",movie-4662,Little Big Top,2006-01-01,NaN
1,A ten year old girl who wanders away from her ...,8.3,125,"[-0.03531729429960251, 0.018258752301335335, 0...",movie-2294,Spirited Away,2001-07-20,The tunnel led Chihiro to a mysterious town...
2,A word for word depiction of the life of Jesus...,8.2,125,"[0.0018316397909075022, 0.023150496184825897, ...",movie-2947,The Visual Bible: The Gospel of John,2003-09-11,For God loved the world So much...
3,"When Sophie, a shy young woman, is cursed with...",8.2,119,"[-0.02675512619316578, 0.015819227322936058, 0...",movie-1987,Howl's Moving Castle,2004-11-19,The two lived there
4,Batman raises the stakes in his war on crime. ...,8.2,152,"[-0.035192325711250305, 0.005697214975953102, ...",movie-65,The Dark Knight,2008-07-16,Why So Serious?
5,Suffering short-term memory loss after a head ...,8.1,113,"[0.0044687893241643906, 0.05293796584010124, -...",movie-3573,Memento,2000-10-11,Some memories are best forgotten.
6,Cidade de Deus is a shantytown that started du...,8.1,130,"[0.037750665098428726, 0.02725977823138237, -0...",movie-3866,City of God,2002-02-05,"If you run you're dead... if you stay, you're ..."
7,Aragorn is revealed as the heir to the ancient...,8.1,201,"[-0.018520008772611618, 0.052507784217596054, ...",movie-329,The Lord of the Rings: The Return of the King,2003-12-01,The eye of the enemy is moving.
8,"Young hobbit Frodo Baggins, after inheriting a...",8.0,178,"[-0.015187859535217285, 0.020928336307406425, ...",movie-262,The Lord of the Rings: The Fellowship of the Ring,2001-12-18,One ring to rule them all
9,"Seymour Polatkin is a successful, gay Indian p...",8.0,103,"[0.011711165308952332, 0.0112167252227664, -0....",movie-4678,The Business of Fancydancing,2002-01-14,Sometimes going home is the hardest journey of...


#### 6) **다양한 LLM 모델 활용** 

- cypher 쿼리 생성하는 모델과 최종 답변 생성 모델을 다르게 적용

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Cypher 쿼리 생성은 OpenAI 모델, 최종 답변은 Gemini 모델 사용
cypher_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.0)
qa_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0)

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=cypher_llm,  # Cypher 쿼리 생성용 LLM
    qa_llm=qa_llm,          # 최종 답변 생성용 LLM
    graph=graph, 
    allow_dangerous_requests=True,
    verbose=True,
)

# 특정 배우와 감독이 함께 작업한 영화 찾기
answer = cypher_chain.invoke({"query": "배우 'Leonardo DiCaprio'와 감독 'Christopher Nolan'이 함께 작업한 영화가 있나요?"})

In [ ]:
for k, v in answer.items():
    print(f"{k}: {v}")

#### 7) **커스텀 프롬프트 활용**

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate

# Cypher 생성을 위한 영화 데이터베이스 특화 프롬프트
CYPHER_GENERATION_TEMPLATE = """Task: 영화 그래프 데이터베이스에 질의하기 위한 Cypher 구문을 생성합니다.
Instructions:
- 스키마에 제공된 노드 레이블, 관계 유형, 속성만 사용하십시오.
- 스키마에 명시되지 않은 관계 유형이나 속성은 사용하지 마십시오.
- 영화 데이터에서 **의미 있는 통찰(meaningful insights)**을 추출하는 데 집중하십시오.

Schema:
{schema}

Note: 
- 제공된 Cypher 구문만 제공할 것.
- 설명이나 사과는 포함하지 말 것.
- 정확하고 관련성 높은 Cypher 쿼리를 생성할 것.

Examples:
# 특정 배우가 출연한 영화 찾기
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
RETURN m.title, m.released, m.rating
ORDER BY m.released DESC

# 특정 장르의 평점 높은 영화 찾기
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)
WHERE g.name = 'Action'
RETURN m.title, m.rating
ORDER BY m.rating DESC
LIMIT 10

# 특정 감독의 영화 중 가장 흥행한 작품
MATCH (p:Person)-[:DIRECTED]->(m:Movie)
WHERE p.name = 'Steven Spielberg'
RETURN m.title, m.rating
ORDER BY m.rating DESC
LIMIT 5

# 특정 연도에 개봉한 영화 조회
MATCH (m:Movie)
WHERE m.released = 2000
RETURN m.title, m.rating
ORDER BY m.rating DESC

# 특정 배우와 감독이 함께 작업한 영화 찾기
MATCH (actor:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(director:Person)
WHERE actor.name = 'Leonardo DiCaprio' AND director.name = 'Christopher Nolan'
RETURN m.title, m.released, m.rating

The question is:
{question}"""

# 결과 처리를 위한 영화 QA 프롬프트
QA_TEMPLATE = """
당신은 영화 데이터베이스 분석 전문가로, 영화 정보에 대한 명확하고 간결한 정보를 한국어로 제공합니다.

[질문]
{question}

[검색 결과]
{context}

# 응답 가이드라인:
- 검색 결과에서 핵심 정보를 요약하세요
- 영화 데이터에 대한 명확하고 객관적인 개요를 제공하세요
- 전문적이고 유익한 톤을 사용하세요
- 영화 데이터에서 중요한 패턴이나 트렌드를 강조하세요
- 맥락이 불충분한 경우 더 많은 정보가 필요하다고 명확히 언급하세요
- 추측이나 개인적인 해석은 피하세요

# 응답 형식:
- 간략한 발견 요약으로 시작하세요
- 여러 영화가 발견된 경우 간결한 개요를 제공하세요
- 가독성을 위해 글머리 기호나 짧은 단락을 사용하세요
- 개봉일, 평점, 주요 배우나 감독과 같은 관련 세부 정보를 포함하세요
- 모든 숫자 데이터나 기술 용어를 이해하기 쉬운 언어로 번역하세요

# 예시 응답 구조:
"분석 결과, [주요 발견 요약]

주요 특징:
- [첫 번째 중요 인사이트]
- [두 번째 중요 인사이트]

추가 정보: [필요한 경우 추가 설명]"
"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

QA_PROMPT = PromptTemplate(
    input_variables=["question", "context"], 
    template=QA_TEMPLATE
)

# Chain 생성 - 주목: input_key와 output_key를 명시적으로 설정
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    # cypher_llm=llm,
    # qa_llm=qa_llm,
    graph=graph, 
    allow_dangerous_requests=True,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    qa_prompt=QA_PROMPT,
    input_key="question",  
    output_key="result"
)

# Cypher 쿼리 실행
answer = cypher_chain.invoke({"question": "배우 'Leonardo DiCaprio'와 감독 'Christopher Nolan'이 함께 작업한 영화가 있나요?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (actor:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(director:Person)
WHERE actor.name = 'Leonardo DiCaprio' AND director.name = 'Christopher Nolan'
RETURN m.title, m.released, m.rating
Full Context:
[{'m.title': 'Inception', 'm.released': '2010-07-14', 'm.rating': 8.1}]

> Finished chain.


In [57]:
for k, v in answer.items():
    print(f"{k}: {v}")

question: 배우 'Leonardo DiCaprio'와 감독 'Christopher Nolan'이 함께 작업한 영화가 있나요?
result: 분석 결과, 배우 레오나르도 디카프리오(Leonardo DiCaprio)와 감독 크리스토퍼 놀란(Christopher Nolan)이 함께 작업한 영화는 2010년에 개봉한 '인셉션(Inception)' 한 편으로 확인되었습니다.

주요 특징:
- 영화 제목: 인셉션 (Inception)
- 개봉일: 2010년 7월 14일
- 평점: 10점 만점 기준 약 8.1점으로 높은 평가를 받음
- 해당 작품은 두 인물이 협업한 대표작으로, 복잡한 스토리와 시각 효과가 돋보이는 작품임

추가 정보: 현재 제공된 데이터에서는 두 인물이 함께 작업한 다른 영화는 확인되지 않았으며, 더 많은 정보를 원할 경우 추가 데이터 검색이 필요합니다.
